In [1]:
#! /usr/bin/env python3
import sys
import time
from utils import get_boot_sector, get_fat_info, get_file_system, get_directory, get_linked_blocks

In [19]:
def write_file_to_disk(f, file_data, fat, block_size, root_dir_start, root_dir_blocks):
    # Calculate the number of blocks needed
    num_blocks_needed = (len(file_data) + block_size - 1) // block_size

    # Find free blocks
    free_blocks = [k for k, v in fat.items() if v == 0x00][:num_blocks_needed]
    # print(free_blocks, num_blocks_needed)
    if len(free_blocks) < num_blocks_needed:
        raise Exception("Not enough free space on the disk")

    # Write data to the blocks
    for i, block in enumerate(free_blocks):
        f.seek(block * block_size)
        # print(f"Going to write at block {block}", file_data[i * block_size:(i + 1) * block_size])
        f.write(file_data[i * block_size:(i + 1) * block_size])

        # Update FAT in-memory (linking blocks)
        if i < len(free_blocks) - 1:
            fat[block] = free_blocks[i + 1]
        else:
            # Mark the last block in the FAT
            fat[block] = 0xFFFFFFFE  # End of file marker
    
    return free_blocks[0]

def update_fat(f, fat, fat_start, block_size):
    f.seek(fat_start * block_size)
    for entry in fat.values():
        # print(entry.to_bytes(4, byteorder='big'))
        f.write(entry.to_bytes(4, byteorder='big'))

def update_directory(f, destination_path, file_data, start_block, block_size, file_system, root_dir_start, root_dir_blocks):
    # Split the destination path to get directory and filename
    path_parts = destination_path.split('/')
    dir_path = '/'.join(path_parts[:-2])
    the_dir = path_parts[-2]
    file_name = path_parts[-1]

    # Find the directory entry
    if dir_path == '':
        dir_path = '/'

        print(file_system[dir_path]['entries'])
        directory_entry = {'start_block': root_dir_start, 'num_blocks': root_dir_blocks, 'file_name': '/'}
    else:
        directory_entry = [x for x in file_system[dir_path]['entries'] if x['file_name'] == the_dir][0]

    

    f.seek(directory_entry['start_block'] * block_size)
    status = -1
    while status != 0:
        status = int.from_bytes(f.read(1), 'big')
        if status != 0:
            f.read(63)

    # seek back one byte, so we put the file entry in the right place, from current spot
    f.seek(-1, 1)
    
    # write in a 3 for status 
    f.write(b'\x03')

    f.write(start_block.to_bytes(4, byteorder='big'))

    num_blocks = (len(file_data) + block_size - 1) // block_size
    f.write(num_blocks.to_bytes(4, byteorder='big'))

    file_size = len(file_data)
    f.write(file_size.to_bytes(4, byteorder='big'))

    current_time = time.localtime()
    year, month, day, hour, minute, second = current_time.tm_year, current_time.tm_mon, current_time.tm_mday, current_time.tm_hour, current_time.tm_min, current_time.tm_sec
    # Write creation times
    f.write(year.to_bytes(2, byteorder='big'))
    f.write(month.to_bytes(1, byteorder='big'))
    f.write(day.to_bytes(1, byteorder='big'))
    f.write(hour.to_bytes(1, byteorder='big'))
    f.write(minute.to_bytes(1, byteorder='big'))
    f.write(second.to_bytes(1, byteorder='big'))
    # Modification times
    f.write(year.to_bytes(2, byteorder='big'))
    f.write(month.to_bytes(1, byteorder='big'))
    f.write(day.to_bytes(1, byteorder='big'))
    f.write(hour.to_bytes(1, byteorder='big'))
    f.write(minute.to_bytes(1, byteorder='big'))
    f.write(second.to_bytes(1, byteorder='big'))

    filename_bytes = file_name.encode('utf-8')
    # Ensure that the filename does not exceed 31 bytes
    if len(filename_bytes) > 31:
        raise ValueError("Filename is too long.")
    null_bytes_needed = 31 - len(filename_bytes)

    f.write(filename_bytes)
    f.write(b'\x00' * null_bytes_needed)

    filler = b'\xFF' * 6
    f.write(filler)

In [20]:
image_file = '../test.img'
source_path = './syyy.txt'
destination_path = '/syyy.txt'

In [21]:

with open(image_file, "r+b") as f, open(source_path, "rb") as source_file:
    os_name, block_size, block_count, fat_start, fat_blocks, root_dir_start, root_dir_blocks = get_boot_sector(f)
    fat = get_fat_info(f, fat_start, fat_blocks, block_size)
    file_data = source_file.read()
    file_system = get_file_system(f, root_dir_blocks, block_size, fat, root_dir_start)
    # WRITE THE FILE INTO THE BLOCKS
    # to implement
    start_block = write_file_to_disk(f, file_data, fat, block_size, root_dir_start, root_dir_blocks)
    
    # 2. UPDATE THE FAT
    # to implement
    update_fat(f, fat, fat_start, block_size) 
    
    # 3. UPDATE THE DIRECTORY
    # to implement
    update_directory(f, destination_path, file_data, start_block, block_size, file_system, root_dir_start, root_dir_blocks) 

[{'status': 3, 'file_name': 'mkfile.cc', 'start_block': 1, 'num_blocks': 2, 'file_size': 735, 'creation_time': '2005/11/15 12:00:00', 'modification_time': 2204568310775808}, {'status': 3, 'file_name': 'foo.txt', 'start_block': 61, 'num_blocks': 5, 'file_size': 2560, 'creation_time': '2005/11/15 12:00:00', 'modification_time': 2204568310775808}, {'status': 3, 'file_name': 'disk.img.gz', 'start_block': 66, 'num_blocks': 8, 'file_size': 3940, 'creation_time': '2009/08/04 21:11:13', 'modification_time': 2208953288428301}]
time: 2023/11/26 23:29:57
file_name syyy.txt


In [48]:
with open(image_file, "r+b") as f, open(source_path, "rb") as source_file:
    update_directory(f, destination_path, file_data, start_block, block_size, file_system)

time: 2023/11/26 22:51:11
file_name syyy.txt


In [49]:
with open(image_file, 'rb') as f:
    blocks = get_linked_blocks(f, 4, block_size, 84, fat)
    entries = get_directory(blocks, block_size, root_dir_blocks)

entries

[{'status': 3,
  'file_name': 'foo.txt',
  'start_block': 61,
  'num_blocks': 5,
  'file_size': 2560,
  'creation_time': '2005/11/15 12:00:00',
  'modification_time': 2204568310775808},
 {'status': 120,
  'file_name': 't                       ',
  'start_block': 50331648,
  'num_blocks': 25600,
  'file_size': 13090917,
  'creation_time': '25655/84/101 100:55:84',
  'modification_time': 32503184952816760},
 {'status': 3,
  'file_name': 'xt                       ',
  'start_block': 2013462528,
  'num_blocks': 100,
  'file_size': 51136,
  'creation_time': '25956/55/179 101:100:55',
  'modification_time': 50510986397429364},
 {'status': 3,
  'file_name': 'xt                       ',
  'start_block': 888,
  'num_blocks': 100,
  'file_size': 51136,
  'creation_time': '25956/55/234 101:100:55',
  'modification_time': 65992110116515444},
 {'status': 3,
  'file_name': 'syyy.txt                       ',
  'start_block': 888,
  'num_blocks': 100,
  'file_size': 51136,
  'creation_time': '2023/11/

In [43]:
file_name

NameError: name 'file_name' is not defined